# Fixations to Bottom-Strip
Determine the value of hyperparameter `cnfg.FIXATIONS_TO_STRIP_THRESHOLD`, which is used to decide if a fixation / visit is a _LWS_ instance or not.

In [8]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

import config as cnfg
from data_models.SearchArray import SearchArray

# pio.renderers.default = "notebook"
pio.renderers.default = "browser"

### Read data

In [3]:
from analysis.pipeline.full_pipeline import read_saved_data

_targets, actions, _metadata, idents, fixations, _visits = read_saved_data()

In [23]:
fixs_with_ident_time = fixations.copy()
dva_cols = [col for col in fixations.columns if col.endswith("distance_dva")]
fixs_with_ident_time["target"] = fixs_with_ident_time[dva_cols].idxmin(axis=1).str.replace("_distance_dva", "")
fixs_with_ident_time = (
    fixs_with_ident_time
    .drop(columns=[col for col in fixs_with_ident_time.columns if "_distance_" in col])
    .merge(
        idents.loc[
            idents["identification_category"] == "hit", ["subject", "trial", "target", "time"]
        ], on=["subject", "trial", "target"], how="left"
    )
)

fixs_with_ident_time.loc[:, "is_during"] = (fixs_with_ident_time["start_time"] <= fixs_with_ident_time["time"]) & (fixs_with_ident_time["time"] <= fixs_with_ident_time["end_time"])

fixs_with_ident_time.loc[:, "is_in_strip"] = fixs_with_ident_time.apply(
    lambda row: SearchArray.is_in_bottom_strip(tuple([row["x"], row["y"]])), axis=1
)

##### Count how many fixations are between a bottom-strip fixation and a target-identification fixation:

In [42]:
def count_fixations_from_strip_to_identification(is_in_strip: pd.Series, is_during: pd.Series) -> pd.Series:
    """
    For a single trial's fixations, count how many fixations are between a bottom-strip fixation and a target-identification fixation.
    For non-bottom-strip fixations, the count is NaN.
    """
    assert len(is_in_strip) == len(is_during), "is_in_strip and is_during must have the same length"
    assert is_in_strip.index.equals(is_during.index), "is_in_strip and is_during must have the same index"
    num_fixs_between = pd.Series(np.nan, index=is_during.index, dtype=float)
    during_indices = np.flatnonzero(is_during.values)
    if len(during_indices) == 0:
        return num_fixs_between     # no identification fixations - return NaN for all fixations
    row_indices = np.arange(len(is_during))
    next_during_indices = np.searchsorted(during_indices, row_indices, side="right")
    has_subsequent_ident = next_during_indices < len(during_indices)
    ident_distances = np.full_like(is_during, np.nan, dtype=float)
    ident_distances[has_subsequent_ident] = during_indices[next_during_indices[has_subsequent_ident]] - row_indices[has_subsequent_ident]    # distance between each fixation and the next identification fixation
    num_fixs_between[is_in_strip] = ident_distances[is_in_strip]    # apply only to bottom-strip fixations
    return num_fixs_between


# populate the dataframe
fixs_with_ident_time["num_fixs_strip_to_ident"] = np.nan
for (subj_id, trial_num, eye), data in fixs_with_ident_time.groupby(["subject", "trial", "eye"]):
    data = data.sort_values("start_time")
    num_fixs_to_strip = count_fixations_from_strip_to_identification(
        data["is_in_strip"], data["is_during"]
    )
    fixs_with_ident_time.loc[data.index, "num_fixs_strip_to_ident"] = num_fixs_to_strip

### Number of Fixations until Bottom-Strip Visit
#### (1) All Fixations

In [6]:
valid_fixs_with_ident_time = fixs_with_ident_time.loc[np.isfinite(fixs_with_ident_time["num_fixs_to_strip"])]

fixs_summary = (
    pd.concat([
        valid_fixs_with_ident_time["num_fixs_to_strip"].describe().rename("all"),
        valid_fixs_with_ident_time.groupby("subject")["num_fixs_to_strip"].describe().T,
    ], axis=1)
).T

print("All Fixations:")
fixs_summary

All Fixations:


,count,mean,std,min,25%,50%,75%,max
all,107523.0,25.675530,24.335446,0.0,7.0,18.0,37.0,195.0
1,4407.0,16.783753,13.674354,0.0,6.0,14.0,25.0,64.0
2,9852.0,38.810191,32.393688,0.0,13.0,31.0,57.0,180.0
3,9674.0,26.289022,22.808377,0.0,8.0,20.0,38.0,121.0
12,9416.0,26.299915,25.405702,0.0,7.0,18.0,37.0,144.0
13,7639.0,20.379369,18.225862,0.0,6.0,15.0,30.0,118.0
14,8704.0,28.407858,27.306917,0.0,8.0,20.0,41.0,195.0
15,6549.0,18.349519,18.144389,0.0,5.0,13.0,26.0,103.0
16,8666.0,23.915532,20.960354,0.0,8.0,18.0,34.0,100.0
17,4658.0,23.375698,22.839839,0.0,7.0,17.0,32.0,153.0


#### (2) Ident Fixations

In [7]:
ident_fixs = fixs_with_ident_time.loc[fixs_with_ident_time["is_during"]]
valid_ident_fixs_with_ident_time = ident_fixs.loc[np.isfinite(fixs_with_ident_time["num_fixs_to_strip"])]

fixs_summary = (
    pd.concat([
        valid_ident_fixs_with_ident_time["num_fixs_to_strip"].describe().rename("all"),
        valid_ident_fixs_with_ident_time.groupby("subject")["num_fixs_to_strip"].describe().T,
    ], axis=1)
).T

print("Identification Fixations:")
fixs_summary

Identification Fixations:


,count,mean,std,min,25%,50%,75%,max
all,1692.0,21.233452,23.700736,1.0,4.00,13.0,29.00,129.0
1,84.0,14.166667,14.935874,1.0,3.00,8.0,20.25,63.0
2,111.0,33.135135,31.748010,1.0,9.00,24.0,44.00,128.0
3,170.0,22.364706,21.503734,1.0,8.00,16.5,29.75,116.0
12,105.0,24.247619,25.622549,1.0,5.00,15.0,31.00,120.0
13,138.0,19.413043,20.491311,1.0,4.25,13.0,27.00,117.0
14,137.0,17.065693,24.147891,1.0,2.00,9.0,19.00,129.0
15,119.0,11.957983,16.405040,1.0,2.00,5.0,16.50,90.0
16,139.0,23.935252,21.635052,1.0,11.00,16.0,32.00,93.0
17,74.0,19.283784,16.763071,1.0,7.00,14.0,28.00,63.0


### Number of Fixations from Bottom-Strip to Identification

In [49]:
finite_num_from_strip_to_iden = fixs_with_ident_time.loc[np.isfinite(fixs_with_ident_time["num_fixs_strip_to_ident"])]

percentiles = [0.01, 0.05, 0.1, 0.25, 0.5]
fixs_summary = (
    pd.concat([
        finite_num_from_strip_to_iden["num_fixs_strip_to_ident"].describe(percentiles).rename("all"),
        finite_num_from_strip_to_iden.groupby("subject")["num_fixs_strip_to_ident"].describe(percentiles).T,
    ], axis=1)
).T

print("From Strip to Identification:")
fixs_summary

From Strip to Identification:


,count,mean,std,min,1%,5%,10%,25%,50%,max
all,2775.0,19.636396,17.108059,1.0,1.0,2.0,3.0,6.0,15.0,127.0
1,93.0,10.311828,6.450444,1.0,1.0,2.0,2.0,4.0,10.0,23.0
2,131.0,23.190840,19.405437,1.0,1.0,2.0,3.0,7.0,19.0,76.0
3,219.0,18.073059,14.111098,1.0,1.0,2.0,3.0,7.0,15.0,68.0
12,211.0,25.090047,24.650634,1.0,2.0,3.0,4.0,7.0,15.0,127.0
13,225.0,15.106667,13.599081,2.0,2.0,2.0,2.0,3.0,10.0,64.0
14,191.0,20.115183,14.108502,1.0,1.9,2.0,4.0,10.0,17.0,62.0
15,279.0,18.551971,16.034697,1.0,1.0,2.0,3.0,6.0,13.0,70.0
16,240.0,20.729167,17.394439,1.0,1.0,2.0,3.0,6.0,16.0,70.0
17,120.0,16.575000,15.145048,1.0,1.0,2.0,2.0,3.0,12.0,62.0
